<a href="https://colab.research.google.com/github/relew/matchPrediction/blob/master/Prediction_final_Zoli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
url = 'http://dataevents.hu/202303/filedataevents/competition_table.xlsx'

In [3]:
data = pd.read_excel(url)

In [4]:
data.columns = data.columns.str.replace("yelllow", "yellow")

In [5]:
data.head()

,match_id,season,div,odds_home_team_win,odds_draw,odds_away_team_win,home_team_match_nr,home_team_goal_roll1_sum,home_team_goal_roll2_sum,home_team_goal_roll3_sum,...,odds_away_team_draw_roll4_mean,odds_away_team_defeat_roll1_mean,odds_away_team_defeat_roll2_mean,odds_away_team_defeat_roll3_mean,odds_away_team_defeat_roll4_mean,away_team_point_mean,away_team_expected_point_mean,home_win_flag,draw_flag,away_win_flag
0,0,2008-2009,div0,1.73,3.5,5.00,5,3,7,8,...,3.825,11.00,7.665,6.377,5.908,0.500,3.0,0,0,1
1,1,2008-2009,div0,1.25,5.5,12.00,7,0,2,5,...,3.582,2.10,2.250,2.833,2.475,0.333,0.0,0,1,0
2,2,2008-2009,div0,1.57,3.8,5.75,9,1,4,4,...,3.525,6.75,4.275,3.883,3.638,0.750,1.5,1,0,0
3,3,2008-2009,div0,1.17,7.0,15.00,11,2,6,7,...,3.975,4.20,2.750,2.333,2.550,0.600,0.0,1,0,0
4,4,2008-2009,div0,1.17,7.0,15.00,14,2,4,7,...,3.925,3.10,2.175,2.050,2.015,0.833,0.0,1,0,0


In [6]:
data["Win_draw_diff"] = data["odds_home_team_win"]-data["odds_draw"]
data['odds_diff'] = data["odds_home_team_win"]-data["odds_away_team_win"]

In [7]:
all_cols = []
for item in data.columns:
  if "_roll" in item:
    value = item.split("_roll")[0]
    if value not in all_cols:
      all_cols.append(value)

In [8]:
def col_creator(data):
  used = []
  for item in data.columns:
    if "sum" in item:
      val = item.split("_roll")[0]
      if (val not in used) & ("sum" in item):
        used.append(val)
        color1 = val+"_roll1_sum"
        color2 = val+"_roll2_sum"
        color3 = val+"_roll3_sum"
        color4 = val+"_roll4_sum"
        sec = "second_match_"+val
        third = "third_match_"+val
        four = "forth_match_"+val
        data[sec] = data[color2] - data[color1] 
        data[third] = data[color3] - data[color1] - data[sec]
        data[four] = data[color4] - data[color1] - data[sec] - data[third]
      elif (val not in used) & ("mean" in item):
        color1 = val+"_roll1_mean"
        color2 = val+"_roll2_mean"
        color3 = val+"_roll3_mean"
        color4 = val+"_roll4_mean"
        sec = "second_match_"+val
        third = "third_match_"+val
        four = "forth_match_"+val
        data[sec] = data[color2] * 2 - data[color1] 
        data[third] = data[color3] * 3 - data[color1] - data[sec]
        data[four] = data[color4] * 4 - data[color1] - data[sec] - data[third]
  return data

In [9]:
data = col_creator(data)

<ipython-input-8-a75f880564cf>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[four] = data[color4] - data[color1] - data[sec] - data[third]


In [10]:
for item in data.columns:
  if data.columns.isnull().sum() != 0:
    print(item)

In [11]:
# Seeing where team is located on table based on str....3 mean top team, 2 middle, 1 relegation pos....change this to based on avg points scored
data["home_team_pos_str"] = np.where(data["home_team_point_mean"] > 1.8, 3, np.where(data["home_team_point_mean"] < 1.2, 1, 2))
data["away_team_pos_str"] = np.where(data["away_team_point_mean"] > 1.8, 3, np.where(data["away_team_point_mean"] < 1.2, 1, 2))

<ipython-input-11-e6bca26a691e>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data["home_team_pos_str"] = np.where(data["home_team_point_mean"] > 1.8, 3, np.where(data["home_team_point_mean"] < 1.2, 1, 2))
<ipython-input-11-e6bca26a691e>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data["away_team_pos_str"] = np.where(data["away_team_point_mean"] > 1.8, 3, np.where(data["away_team_point_mean"] < 1.2, 1, 2))


In [12]:
sum_cols = []
for item in data.columns:
  if "sum" in item:
    value = item.split("_roll")[0]
    if value not in sum_cols:
      sum_cols.append(value)

In [13]:
sum_cols

['home_team_goal',
 'home_team_opponents_goal',
 'home_team_shot',
 'home_team_opponents_shot',
 'home_team_shot_on_target',
 'home_team_opponents_shot_on_target',
 'home_team_fault',
 'home_team_opponents_fault',
 'home_team_corner',
 'home_team_opponents_corner',
 'home_team_yellow_card',
 'home_team_opponents_yellow_card',
 'home_team_red_card',
 'home_team_opponents_red_card',
 'home_team_point',
 'home_team_expected_point',
 'away_team_goal',
 'away_team_opponents_goal',
 'away_team_shot',
 'away_team_opponents_shot',
 'away_team_shot_on_target',
 'away_team_opponents_shot_on_target',
 'away_team_fault',
 'away_team_opponents_fault',
 'away_team_corner',
 'away_team_opponents_corner',
 'away_team_yellow_card',
 'away_team_opponents_yellow_card',
 'away_team_red_card',
 'away_team_opponents_red_card',
 'away_team_point',
 'away_team_expected_point']

In [14]:
def medianer(X):
  for item in sum_cols:
    colname = item+"_median"
    val1 = item+"_roll1_sum"
    val2 = "second_match_"+item
    val3 ="third_match_"+item
    val4 ="forth_match_"+item
    X[colname] = np.sort(X, axis=1)[:, 1:3].sum(1)/2
  return X

In [15]:
for item in sum_cols:
  colname = item+"_median"
  columns = [c for c in data.columns if item in c]
  data[colname] = data[columns].apply(lambda x: sorted(x)[1:3], axis=1, result_type='expand').sum(1)/2


In [16]:
def wincounter(data):
  if data["home_team_goal_roll1_sum"] > data["home_team_opponents_goal_roll1_sum"]:
    if data["second_match_home_team_goal"] > data["second_match_home_team_opponents_goal"]:
      if data["third_match_home_team_goal"] > data["third_match_home_team_opponents_goal"]:
        if data["forth_match_home_team_goal"] > data["forth_match_home_team_opponents_goal"]:
          return 4
        else: 
          return 3
      else:
        return 2
    else:
      return 1
  else:
    return 0

def aw_wincounter(data):
  if data["away_team_goal_roll1_sum"] > data["away_team_opponents_goal_roll1_sum"]:
    if data["second_match_away_team_goal"] > data["second_match_away_team_opponents_goal"]:
      if data["third_match_away_team_goal"] > data["third_match_away_team_opponents_goal"]:
        if data["forth_match_away_team_goal"] > data["forth_match_away_team_opponents_goal"]:
          return 4
        else: 
          return 3
      else:
        return 2
    else:
      return 1
  else:
    return 0

In [17]:
data["home_win_streak"] = data.apply(lambda x: wincounter(x), axis = 1)

In [18]:
data["away_win_streak"] = data.apply(lambda x: aw_wincounter(x), axis = 1)

In [19]:
data["home_win_streak"].value_counts()

0    43376
1    11991
2     4968
4     1943
3     1888
Name: home_win_streak, dtype: int64

In [20]:
data["away_win_streak"].value_counts()

0    37675
1    16584
2     5415
3     2342
4     2150
Name: away_win_streak, dtype: int64

In [21]:
clean_cols = []
for item in sum_cols:
  val = item.split("_team_")[1]
  if ("opponents_" not in val) & (val not in clean_cols):
    clean_cols.append(val)

In [22]:
clean_cols

['goal',
 'shot',
 'shot_on_target',
 'fault',
 'corner',
 'yellow_card',
 'red_card',
 'point',
 'expected_point']

In [23]:
def ratio(data):
  for item in clean_cols:
    home_val = "home_team_" + item + "_median"
    away_val = "away_team_" + item + "_median"
    rat = item+"_ratio"
    data[rat] = data[home_val] / data[away_val]
  return data

def pre_match_ratio(data):
  for item in clean_cols[:-2]:
    home_val = "home_team_" + item + "_median"
    home_opp_val = "home_team_opponents_" + item + "_median"
    rat = item+"home_ratio"
    data[rat] = data[home_val] / data[home_opp_val]
    away_val = "away_team_" + item + "_median"
    away_opp_val = "away_team_opponents_" + item + "_median"
    rat_aw = item+"away_ratio"
    data[rat_aw] = data[away_val] / data[away_opp_val]
  return data




In [24]:
data = pre_match_ratio(data)

In [25]:
cols_use = []

for item in data.columns:
  if ("ratio" in item) & ("opponents" not in item):
    cols_use.append(item)
  elif ("median" in item) & ("opponents" not in item):
    cols_use.append(item)

In [26]:
cols_use

['home_team_goal_median',
 'home_team_shot_median',
 'home_team_shot_on_target_median',
 'home_team_fault_median',
 'home_team_corner_median',
 'home_team_yellow_card_median',
 'home_team_red_card_median',
 'home_team_point_median',
 'home_team_expected_point_median',
 'away_team_goal_median',
 'away_team_shot_median',
 'away_team_shot_on_target_median',
 'away_team_fault_median',
 'away_team_corner_median',
 'away_team_yellow_card_median',
 'away_team_red_card_median',
 'away_team_point_median',
 'away_team_expected_point_median',
 'goalhome_ratio',
 'goalaway_ratio',
 'shothome_ratio',
 'shotaway_ratio',
 'shot_on_targethome_ratio',
 'shot_on_targetaway_ratio',
 'faulthome_ratio',
 'faultaway_ratio',
 'cornerhome_ratio',
 'corneraway_ratio',
 'yellow_cardhome_ratio',
 'yellow_cardaway_ratio',
 'red_cardhome_ratio',
 'red_cardaway_ratio']

In [27]:
data["Pos_diff"] = data["home_team_pos_str"] - data["away_team_pos_str"]

In [28]:
# good 67.42
data["home_team_roll4_points_diff"] = data["home_team_point_roll4_sum"]-data["home_team_expected_point_roll4_sum"]
data["away_team_roll4_points_diff"] = data["away_team_point_roll4_sum"]-data["away_team_expected_point_roll4_sum"]
# good 67.46
data["home_team_roll1_points_diff"] = data["home_team_point_roll1_sum"]-data["home_team_expected_point_roll1_sum"]
data["away_team_roll1_points_diff"] = data["home_team_point_roll1_sum"]-data["home_team_expected_point_roll1_sum"]

In [29]:
df = data[['odds_home_team_win','odds_away_team_win','odds_draw',"Pos_diff",'odds_diff','Win_draw_diff', 'home_team_pos_str', 'away_team_pos_str', 'home_team_shot_median', 'home_team_shot_on_target_median', 'home_team_corner_median','away_team_goal_median',
 'away_team_shot_median','away_team_shot_on_target_median','away_team_corner_median','away_team_red_card_roll1_sum','shothome_ratio',
 'shotaway_ratio',
 'shot_on_targethome_ratio',
 'shot_on_targetaway_ratio','cornerhome_ratio','corneraway_ratio', "home_win_flag"]]

In [30]:
#'home_team_fault_median','home_team_yellow_card_median','home_team_red_card_roll1_sum','away_team_fault_median','away_team_yellow_card_median','faulthome_ratio','faultaway_ratio','yellow_cardhome_ratio','yellow_cardaway_ratio','home_team_roll1_points_diff','away_team_roll1_points_diff','home_team_roll4_points_diff',

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score

In [32]:
model = GaussianNB()

In [33]:
for item in df.columns:
  maxi = df[item].max()
  df[item] = df[item].replace([np.inf, -np.inf], -200000)

<ipython-input-33-7ed1d7d1e8b1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[item] = df[item].replace([np.inf, -np.inf], -200000)


In [34]:
for item in df.columns:
  val = df[item].apply(np.isinf).sum()
  maxi = df[item].max()
  df[item] = df[item].replace(-200000, maxi)

<ipython-input-34-61f9dd506e39>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[item] = df[item].replace(-200000, maxi)


In [35]:
df = df.fillna(0)

In [36]:
y = df["home_win_flag"]
X = df.drop("home_win_flag", axis = 1)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [38]:
model.fit(X_train, y_train)

GaussianNB()

In [39]:
y_proba = model.predict_proba(X_test)[:, 1]

In [40]:
auc = roc_auc_score(y_test, y_proba)

In [41]:
auc

0.6606260249830056

In [42]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

In [43]:
rf = RandomForestClassifier(random_state = 42)

In [44]:
grid = GridSearchCV(rf, param_grid = param_grid, cv = 5)

In [45]:
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [5, 10], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]})

In [46]:
print("Best parameters:", grid.best_params_)
print("Best score:", grid.best_score_)

Best parameters: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Best score: 0.6424660526775172


In [47]:
y_pred = grid.best_estimator_.predict_proba(X_test)[:, 1]

In [48]:
from sklearn.metrics import accuracy_score, roc_auc_score

In [49]:
auc = roc_auc_score(y_test, y_pred)

In [50]:
print(auc)

0.6740696245788187


In [51]:
best_rf = grid.best_estimator_

In [52]:
importances = best_rf.feature_importances_

In [53]:
for feature, importance in zip(X.columns, importances):
    print(f'{feature}: {importance}')

odds_home_team_win: 0.1765923314596245
odds_away_team_win: 0.2101498032551366
odds_draw: 0.055821964348186565
Pos_diff: 0.024373977445716902
odds_diff: 0.23385346085015643
Win_draw_diff: 0.19966929519502924
home_team_pos_str: 0.00892179787906359
away_team_pos_str: 0.010705633611062011
home_team_shot_median: 0.005702204951708855
home_team_shot_on_target_median: 0.0017195685216214833
home_team_corner_median: 0.002790279020534312
away_team_goal_median: 0.0036186966894981524
away_team_shot_median: 0.004474347761198636
away_team_shot_on_target_median: 0.0016559548305675443
away_team_corner_median: 0.0022893745579948086
away_team_red_card_roll1_sum: 0.00016365643688360515
shothome_ratio: 0.00859325577940825
shotaway_ratio: 0.01398308646761966
shot_on_targethome_ratio: 0.012278205687789069
shot_on_targetaway_ratio: 0.010742065868363735
cornerhome_ratio: 0.005200161343604177
corneraway_ratio: 0.006700878039231828


**For away prediction**

In [54]:
away = data[['odds_home_team_win','odds_away_team_win','odds_draw',"Pos_diff",'odds_diff','Win_draw_diff', 'home_team_pos_str', 'away_team_pos_str', 'home_team_shot_median', 'home_team_shot_on_target_median', 'home_team_corner_median','away_team_goal_median',
 'away_team_shot_median','away_team_shot_on_target_median','away_team_corner_median','away_team_red_card_roll1_sum','shothome_ratio',
 'shotaway_ratio',
 'shot_on_targethome_ratio',
 'shot_on_targetaway_ratio','cornerhome_ratio','corneraway_ratio', "away_win_flag"]]

In [55]:
for item in away.columns:
  maxi = away[item].max()
  away[item] = away[item].replace([np.inf, -np.inf], -200000)

<ipython-input-55-9cdccbb82b1b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away[item] = away[item].replace([np.inf, -np.inf], -200000)


In [56]:
for item in away.columns:
  val = away[item].apply(np.isinf).sum()
  maxi = away[item].max()
  away[item] = away[item].replace(-200000, maxi)

<ipython-input-56-232d14e1c800>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away[item] = away[item].replace(-200000, maxi)


In [57]:
away = away.fillna(0)

In [58]:
y = away["away_win_flag"]
X = away.drop("away_win_flag", axis = 1)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [60]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

In [61]:
rf = RandomForestClassifier(random_state = 42)
grid = GridSearchCV(rf, param_grid = param_grid, cv = 5)

In [62]:
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [5, 10], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]})

In [63]:
y_pred = grid.best_estimator_.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred)
print(auc)

0.6791869682971363


Draw prediction

In [68]:
draw = data[['odds_home_team_win','odds_away_team_win','odds_draw',"Pos_diff",'odds_diff','Win_draw_diff', 'home_team_pos_str', 'away_team_pos_str', 'home_team_shot_median', 'home_team_shot_on_target_median', 'home_team_corner_median','away_team_goal_median',
 'away_team_shot_median','away_team_shot_on_target_median','away_team_corner_median','away_team_red_card_roll1_sum','shothome_ratio',
 'shotaway_ratio',
 'shot_on_targethome_ratio',
 'shot_on_targetaway_ratio','cornerhome_ratio','corneraway_ratio', "draw_flag"]]

In [69]:
for item in draw.columns:
  maxi = draw[item].max()
  draw[item] = draw[item].replace([np.inf, -np.inf], -200000)

<ipython-input-69-7fa8474eb9e2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  draw[item] = draw[item].replace([np.inf, -np.inf], -200000)


In [70]:
for item in draw.columns:
  val = draw[item].apply(np.isinf).sum()
  maxi = draw[item].max()
  draw[item] = draw[item].replace(-200000, maxi)

<ipython-input-70-f719730546b2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  draw[item] = draw[item].replace(-200000, maxi)


In [71]:
draw = draw.fillna(0)

In [73]:
y = draw["draw_flag"]
X = draw.drop("draw_flag", axis = 1)

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [75]:
rf = RandomForestClassifier(random_state = 42)
grid = GridSearchCV(rf, param_grid = param_grid, cv = 5)

In [76]:
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [5, 10], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]})

In [77]:
y_pred = grid.best_estimator_.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred)
print(auc)

0.564657942520559
